# Local Offline RAG with Ollama

## Overview
This notebook demonstrates building a **completely offline RAG (Retrieval-Augmented Generation)** system using **Ollama** for local LLMs and embeddings.

### 🚀 Benefits of Local RAG:
- **100% Offline**: No internet required after setup
- **Privacy First**: Your documents never leave your machine
- **No API Costs**: Free to run unlimited queries
- **Fast**: No network latency
- **Full Control**: Customize models and parameters

### 📋 Architecture:
```
PDF Documents → Load → Split → Local Embeddings (Ollama) → ChromaDB
                                                                  ↓
User Query → Retrieve Similar Chunks → Local LLM (Ollama) → Answer
```

### 🛠️ Components:
- **Document Loader**: PyPDFLoader
- **Text Splitter**: RecursiveCharacterTextSplitter
- **Embeddings**: Ollama with nomic-embed-text (or embeddinggemma)
- **Vector Store**: ChromaDB (persistent, local)
- **LLM**: Ollama with gemma3:1b
- **Chain**: LangChain Expression Language (LCEL)

---

## 1. Prerequisites & Installation

### Required Software:
1. **Ollama**: Download from https://ollama.ai
2. **Python 3.9+**: Recommended 3.11 or 3.13

### Install Python Packages:

In [1]:
# Install required packages (run this once)
%pip install langchain langchain-core langchain-community langchain-text-splitters
%pip install langchain-ollama langchain-chroma chromadb
%pip install pypdf tiktoken

# Or install from requirements.txt with additional packages:
%pip install -r requirements.txt
%pip install langchain-ollama langchain-chroma chromadb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of langchain-google-genai to determ

  You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.


### Download Ollama Models:

Run these commands in your terminal (if you haven't already):

```bash
# Embedding model (choose one or both)
ollama pull nomic-embed-text    # Recommended: 274 MB
ollama pull embeddinggemma      # Alternative: 621 MB

# LLM for generation
ollama pull gemma3:1b          # Small & fast: 815 MB
```

**Note**: You already have these models downloaded! ✓

## 2. Import Required Libraries

In [7]:
# Standard library imports
import os
import sys
from pathlib import Path

# LangChain Document Loaders
from langchain_community.document_loaders import PyPDFLoader

# LangChain Text Splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Integration
from langchain_ollama import OllamaEmbeddings, ChatOllama

# ChromaDB Vector Store
from langchain_chroma import Chroma

# LangChain Core Components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("✓ All imports successful!")
print("✓ Ready for local offline RAG!")
print(f"\nPython version: {sys.version}")

✓ All imports successful!
✓ Ready for local offline RAG!

Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]


## 3. Verify Ollama Installation

Let's check that Ollama is running and our models are available.

In [8]:
# Check Ollama is running and list available models
!ollama list

NAME                        ID              SIZE      MODIFIED          
gemma3:latest               a2af6cc3eb7f    3.3 GB    22 minutes ago       
mxbai-embed-large:latest    468836162de7    669 MB    25 minutes ago       
bge-m3:latest               790764642607    1.2 GB    59 minutes ago       
nomic-embed-text:latest     0a109f422b47    274 MB    About an hour ago    
llama3.2:latest             a80c4f17acd5    2.0 GB    3 weeks ago          
gemma3:4b                   a2af6cc3eb7f    3.3 GB    3 weeks ago          


In [9]:
# Test Ollama connection with a simple query
print("Testing Ollama connection...\n")

try:
    test_llm = ChatOllama(model="gemma3:4b", temperature=0)
    # bge-m3:latest 
    response = test_llm.invoke("Say 'Hello! I am running locally on your machine!'")
    
    print("✓ Ollama is working!")
    print(f"Response: {response.content}")
    
except Exception as e:
    print(f"✗ Error connecting to Ollama: {e}")
    print("\nMake sure Ollama is running. Try: ollama serve")

Testing Ollama connection...

✓ Ollama is working!
Response: Hello! I am running locally on your machine! 😊



## 4. Load PDF Documents

Load your PDF documents for the RAG system.

In [4]:
# ===== CONFIGURATION: Update this path to your PDF file =====
pdf_path = "attention.pdf"  # Change this to your PDF file path
# =============================================================

# Check if file exists
if not os.path.exists(pdf_path):
    print(f"⚠️  ERROR: File '{pdf_path}' not found!")
    print("Please update the pdf_path variable with your PDF file location.")
else:
    # Load the PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # Display information
    print(f"✓ Loaded {len(documents)} pages from '{pdf_path}'")
    print(f"\n--- First Page Preview ---")
    print(f"Content (first 300 chars): {documents[0].page_content[:300]}...")
    print(f"\nMetadata: {documents[0].metadata}")
    print(f"\nTotal characters: {sum(len(doc.page_content) for doc in documents):,}")

✓ Loaded 15 pages from 'attention.pdf'

--- First Page Preview ---
Content (first 300 chars): Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Par...

Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}

Total characters: 39,587


### Optional: Load Multiple PDFs from a Directory

In [10]:
# Uncomment to load multiple PDFs from a directory

pdf_directory = "./pdfs"
all_documents = []

if os.path.exists(pdf_directory):
    pdf_files = list(Path(pdf_directory).glob("*.pdf"))
    print(f"Found {len(pdf_files)} PDF files\n")
    
    for pdf_file in pdf_files:
        loader = PyPDFLoader(str(pdf_file))
        docs = loader.load()
        all_documents.extend(docs)
        print(f"  ✓ Loaded {len(docs)} pages from {pdf_file.name}")
    
    print(f"\nTotal pages loaded: {len(all_documents)}")
    documents = all_documents  # Use this for the rest of the pipeline

Found 3 PDF files

  ✓ Loaded 15 pages from attention.pdf
  ✓ Loaded 19 pages from rag.pdf
  ✓ Loaded 21 pages from ragsurvey.pdf

Total pages loaded: 55


## 5. Split Documents into Chunks

Break documents into smaller chunks for better retrieval precision.

In [11]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,        # Characters per chunk
    chunk_overlap=128,      # Overlap to maintain context
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Split on paragraphs, then lines, etc.
)

# Split documents
chunks = text_splitter.split_documents(documents)

# Display results
avg_chunk_size = sum(len(chunk.page_content) for chunk in chunks) / len(chunks)

print(f"✓ Split {len(documents)} documents into {len(chunks)} chunks")
print(f"\nAverage chunk size: {avg_chunk_size:.0f} characters")

# Preview chunks
print(f"\n--- Chunk Examples ---")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1} (length: {len(chunk.page_content)} chars):")
    print(f"{chunk.page_content[:200]}...")

✓ Split 55 documents into 267 chunks

Average chunk size: 898 characters

--- Chunk Examples ---

Chunk 1 (length: 986 chars):
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
...

Chunk 2 (length: 944 chars):
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more pa...

Chunk 3 (length: 986 chars):
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Tra...


## 6. Create Embeddings (Primary: Nomic-Embed-Text)

### About Nomic-Embed-Text:
- **Size**: 274 MB
- **Dimensions**: 768
- **Performance**: State-of-the-art for local embeddings
- **Speed**: Fast inference
- **License**: Open source (Apache 2.0)

In [ ]:
# Initialize Ollama Embeddings with nomic-embed-text
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",   # Embeddings model mxbai-embed-large 
    # base_url="http://localhost:11434"  # Default Ollama URL
)

# Test embeddings
print("Testing nomic-embed-text embeddings...\n")
sample_text = "This is a test sentence for embeddings."
sample_embedding = embeddings.embed_query(sample_text)

print(f"✓ Embeddings model: nomic-embed-text")
print(f"✓ Embedding dimension: {len(sample_embedding)}")
print(f"✓ Sample embedding (first 10 values): {sample_embedding[:10]}")
print(f"\nℹ️  Each chunk will be converted to a {len(sample_embedding)}-dimensional vector")
print(f"ℹ️  All processing happens locally on your machine!")

Testing nomic-embed-text embeddings...

✓ Embeddings model: nomic-embed-text
✓ Embedding dimension: 768
✓ Sample embedding (first 10 values): [0.03251289, 0.060831312, -0.1661455, -0.08210702, 0.043301575, -0.025996638, 0.051529177, -0.015186348, -0.008282056, -0.028375432]

ℹ️  Each chunk will be converted to a 768-dimensional vector
ℹ️  All processing happens locally on your machine!


## 7. Alternative: EmbeddingGemma (Optional)

### About EmbeddingGemma:
- **Size**: 621 MB (larger than nomic)
- **Dimensions**: 768
- **Optimized for**: Google Gemma models
- **Use case**: Better alignment with Gemma LLMs

**Uncomment the code below to use embeddinggemma instead:**

In [13]:
# # Alternative: Use embeddinggemma instead
# embeddings = OllamaEmbeddings(
#     model="embeddinggemma:latest"
# )

# # Test embeddings
# print("Testing embeddinggemma embeddings...\n")
# sample_text = "This is a test sentence for embeddings."
# sample_embedding = embeddings.embed_query(sample_text)

# print(f"✓ Embeddings model: embeddinggemma")
# print(f"✓ Embedding dimension: {len(sample_embedding)}")
# print(f"✓ Sample embedding (first 10 values): {sample_embedding[:10]}")

## 8. Create ChromaDB Vector Store

### Why ChromaDB?
- **Local & Persistent**: Stores vectors on disk
- **Python 3.13 Compatible**: Works with latest Python
- **Easy to Use**: Simple API
- **Open Source**: Free and fully featured

**Note**: This step may take a minute as it processes all chunks.

In [14]:
# Create ChromaDB vector store
print(f"Creating ChromaDB vector store from {len(chunks)} chunks...")
print("This may take a minute...\n")

# Set persistent directory
persist_directory = "./chroma_db"

# Create vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name="local_rag_collection"
)

print(f"✓ ChromaDB vector store created successfully!")
print(f"✓ Indexed {len(chunks)} document chunks")
print(f"✓ Stored at: {persist_directory}")
print(f"\nℹ️  Vector store persisted to disk - you can reload it later!")

Creating ChromaDB vector store from 267 chunks...
This may take a minute...

✓ ChromaDB vector store created successfully!
✓ Indexed 267 document chunks
✓ Stored at: ./chroma_db

ℹ️  Vector store persisted to disk - you can reload it later!


### Load Existing Vector Store (Optional)

If you've already created the vector store, you can load it instead:

In [15]:
# # Uncomment to load existing vector store
# persist_directory = "./chroma_db"

# vectorstore = Chroma(
#     persist_directory=persist_directory,
#     embedding_function=embeddings,
#     collection_name="local_rag_collection"
# )

# print(f"✓ Loaded existing vector store from '{persist_directory}'")
# print(f"✓ Collection: local_rag_collection")

## 9. Create Retriever and Test

The retriever finds the most relevant chunks for a given query.

In [16]:
# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",    # Use cosine similarity
    search_kwargs={"k": 4}        # Retrieve top 4 most relevant chunks
)

print("✓ Retriever configured successfully")
print(f"  - Search type: similarity")
print(f"  - Number of documents to retrieve (k): 4")

# Test retrieval
test_query = "What is the main topic of this document?"
print(f"\n--- Retriever Test ---")
print(f"Query: '{test_query}'")

retrieved_docs = retriever.invoke(test_query)

print(f"\nRetrieved {len(retrieved_docs)} documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Content preview: {doc.page_content[:150]}...")
    print(f"  Source: Page {doc.metadata.get('page', 'N/A')}")

✓ Retriever configured successfully
  - Search type: similarity
  - Number of documents to retrieve (k): 4

--- Retriever Test ---
Query: 'What is the main topic of this document?'

Retrieved 4 documents:

Document 1:
  Content preview: tion VII mainly discusses the challenges that RAG currently
faces and its future development directions. At last, the paper
concludes in Section VIII....
  Source: Page 1

Document 2:
  Content preview: Jonathan Berant. Coarse-to-ﬁne question answering for long documents. In Proceedings of the
55th Annual Meeting of the Association for Computational L...
  Source: Page 9

Document 3:
  Content preview: RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit se...
  Source: Page 1

Document 4:
  Content preview: just the summary! topic-aware convolutional neural networks for ex-
treme summarization,” arXiv preprint arXiv:1808.08745 , 2018.
[154] S. Saha, J. A....


## 10. Configure Ollama LLM (Gemma3:1b)

### About Gemma3:1b:
- **Size**: 815 MB
- **Parameters**: 1 billion
- **Speed**: Very fast inference
- **Quality**: Good for most Q&A tasks
- **Memory**: Low RAM usage

**Alternatives**: You can also use llama3.2 (2GB) or deepseek-r1 (4.7GB) for better quality.

In [18]:
# Initialize Ollama LLM
llm = ChatOllama(
    model="gemma3:4b",
    temperature=0,          # Deterministic responses (0 = focused, 1 = creative)
    # num_predict=2000,     # Max tokens to generate
    # top_k=40,             # Top-k sampling
    # top_p=0.9,            # Top-p (nucleus) sampling
)

print("✓ LLM configured successfully")
print(f"  - Model: gemma3:4b (local)")
print(f"  - Temperature: 0 (deterministic)")

# Test LLM
test_response = llm.invoke("Say 'Hello! I am Gemma running locally!'")
print(f"\nLLM Test Response: {test_response.content}")

✓ LLM configured successfully
  - Model: gemma3:4b (local)
  - Temperature: 0 (deterministic)

LLM Test Response: Hello! I am Gemma running locally! 😊



### Try Other Local Models (Optional)

In [19]:
# # Alternative: Use llama3.2 for better quality
# llm = ChatOllama(
#     model="llama3.2:latest",
#     temperature=0
# )

# # Alternative: Use deepseek-r1 for reasoning tasks
# llm = ChatOllama(
#     model="deepseek-r1:latest",
#     temperature=0
# )

## 11. Build RAG Chain (LangChain Expression Language)

Combine retrieval and generation into a single pipeline using LCEL.

In [21]:
# Define prompt template
system_prompt = (
    "You are a helpful assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer based on the context, say that you don't know. "
    "Keep the answer concise and accurate.\n\n"
    "Context: {context}\n\n"
    "Question: {question}"
)

prompt = ChatPromptTemplate.from_template(system_prompt)

# Helper function to format documents
def format_docs(docs):
    """Format retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Build RAG chain using LCEL
rag_chain = (
    {
        "context": retriever | format_docs,  # Retrieve and format docs
        "question": RunnablePassthrough()      # Pass through the question
    }
    | prompt           # Format with prompt template
    | llm              # Generate answer with local LLM
    | StrOutputParser() # Parse output to string
)

print("✓ RAG chain created successfully using LCEL!")
print("\nRAG Pipeline Flow:")
print("  1. User provides a query")
print("  2. Retriever finds top 4 relevant chunks (local ChromaDB)")
print("  3. Chunks are formatted as context")
print("  4. Context + question formatted with prompt template")
print("  5. Local LLM (gemma3:1b) generates answer")
print("  6. Answer parsed and returned")
print("\n🔒 Everything runs locally on your machine!")

✓ RAG chain created successfully using LCEL!

RAG Pipeline Flow:
  1. User provides a query
  2. Retriever finds top 4 relevant chunks (local ChromaDB)
  3. Chunks are formatted as context
  4. Context + question formatted with prompt template
  5. Local LLM (gemma3:1b) generates answer
  6. Answer parsed and returned

🔒 Everything runs locally on your machine!


## 12. Test RAG Pipeline with Example Queries

Let's test our complete local RAG system!

In [22]:
# Example Query 1: General question
query1 = "What is the main topic or contribution of this document?"

print(f"Query: {query1}")
print("\nProcessing locally...\n")

answer = rag_chain.invoke(query1)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

# Show source documents
print("\nSOURCE DOCUMENTS USED:")
print("=" * 80)
retrieved_docs = retriever.invoke(query1)
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Page: {doc.metadata.get('page', 'N/A')}")
    print(f"  Content: {doc.page_content[:200]}...")
    print("-" * 80)

Query: What is the main topic or contribution of this document?

Processing locally...

ANSWER:
This document primarily discusses the RAG (Retrieval-Augmented Generation) research paradigm, its stages (Naive, Advanced, and Modular RAG), and its evolution, including its expansion to multi-modal data like images.


SOURCE DOCUMENTS USED:

Document 1:
  Page: 1
  Content: tion VII mainly discusses the challenges that RAG currently
faces and its future development directions. At last, the paper
concludes in Section VIII.
II. O VERVIEW OF RAG
A typical application of RAG...
--------------------------------------------------------------------------------

Document 2:
  Page: 1
  Content: RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit several limitations.
The development of Advanced RAG...
--------------------------------------------------------------------------------

Document 3:
  Page: 6
 

In [23]:
# Example Query 2: Specific information extraction
query2 = "Can you summarize the key technical contributions or innovations mentioned?"

print(f"Query: {query2}")
print("\nProcessing locally...\n")

answer = rag_chain.invoke(query2)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

Query: Can you summarize the key technical contributions or innovations mentioned?

Processing locally...

ANSWER:
The key innovations include:

*   Replacing RNNs with self-attention.
*   Designing and implementing the first Transformer models.
*   Scaled dot-product attention and multi-head attention.
*   Incorporating external databases to bridge information gaps for LLMs.



In [24]:
# Example Query 3: Your custom question
custom_query = "What specific details are mentioned about the methodology or approach?"

print(f"Query: {custom_query}")
print("\nProcessing locally...\n")

answer = rag_chain.invoke(custom_query)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer)
print("\n" + "=" * 80)

Query: What specific details are mentioned about the methodology or approach?

Processing locally...

ANSWER:
The methodology includes Naive RAG, RAG, Advanced RAG, and Modular RAG. Naive RAG is the earliest methodology, RAG provides a tailored textbook for information retrieval, and Advanced/Modular RAG address limitations of Naive RAG.



## 13. Interactive Q&A Session

Ask your own questions to the RAG system!

In [25]:
# Interactive Q&A
def ask_question(question):
    """Ask a question to the RAG system."""
    print(f"\n{'='*80}")
    print(f"Question: {question}")
    print(f"{'='*80}")
    
    answer = rag_chain.invoke(question)
    
    print(f"\nAnswer: {answer}")
    print(f"{'='*80}\n")
    
    return answer

# Try it out!
# Change the question below to ask anything about your document
my_question = "What are the main findings or results?"
ask_question(my_question)


Question: What are the main findings or results?

Answer: The main evaluation objectives include: Retrieval Quality and Generation Quality. Standard metrics from search engines and information retrieval systems are used to measure retrieval performance, and generation quality is assessed based on faithfulness, relevance, and non-harmfulness.



'The main evaluation objectives include: Retrieval Quality and Generation Quality. Standard metrics from search engines and information retrieval systems are used to measure retrieval performance, and generation quality is assessed based on faithfulness, relevance, and non-harmfulness.'

## 14. Bonus: Compare Embedding Models (Optional)

Compare retrieval results between nomic-embed-text and embeddinggemma.

In [26]:
# # Uncomment to compare embedding models

# print("Comparing embedding models...\n")

# test_query = "What is attention mechanism?"

# # Test with nomic-embed-text
# print("=" * 80)
# print("Using nomic-embed-text:")
# print("=" * 80)
# embeddings_nomic = OllamaEmbeddings(model="nomic-embed-text")
# vectorstore_nomic = Chroma.from_documents(
#     documents=chunks[:10],  # Use first 10 chunks for quick test
#     embedding=embeddings_nomic,
#     collection_name="test_nomic"
# )
# retriever_nomic = vectorstore_nomic.as_retriever(search_kwargs={"k": 2})
# docs_nomic = retriever_nomic.invoke(test_query)

# print(f"\nTop retrieved document:")
# print(f"{docs_nomic[0].page_content[:200]}...\n")

# # Test with embeddinggemma
# print("=" * 80)
# print("Using embeddinggemma:")
# print("=" * 80)
# embeddings_gemma = OllamaEmbeddings(model="embeddinggemma:latest")
# vectorstore_gemma = Chroma.from_documents(
#     documents=chunks[:10],
#     embedding=embeddings_gemma,
#     collection_name="test_gemma"
# )
# retriever_gemma = vectorstore_gemma.as_retriever(search_kwargs={"k": 2})
# docs_gemma = retriever_gemma.invoke(test_query)

# print(f"\nTop retrieved document:")
# print(f"{docs_gemma[0].page_content[:200]}...\n")

# print("=" * 80)
# print("\nℹ️  Both models perform well. Choose based on your preference!")
# print("   - nomic-embed-text: Smaller (274MB), general-purpose")
# print("   - embeddinggemma: Larger (621MB), optimized for Gemma models")

## 15. Performance Tips & Next Steps

### 🚀 Performance Optimization:
1. **Chunk Size**: Experiment with different sizes (512, 1024, 2048)
2. **Retrieval Count (k)**: Try k=3, 4, 5, 6 based on your needs
3. **Model Selection**: 
   - Fast: gemma3:1b
   - Balanced: llama3.2
   - Best quality: deepseek-r1
4. **Temperature**: 0 for factual, 0.3-0.7 for creative

### 💾 Persistence:
- Vector store is saved at `./chroma_db`
- You can reload it without re-embedding documents
- Delete the directory to start fresh

### 🔧 Troubleshooting:
- **Slow responses**: Use smaller model (gemma3:1b)
- **Out of memory**: Reduce chunk count or use smaller model
- **Ollama not found**: Make sure `ollama serve` is running

### 📚 Next Steps:
1. Try different documents and PDFs
2. Experiment with other Ollama models
3. Add custom preprocessing or post-processing
4. Build a simple UI with Gradio or Streamlit
5. Compare with cloud-based RAG (OpenAI, etc.)

### 🎉 Congratulations!
You now have a fully functional **local, offline RAG system** running on your machine!

---

**Created with LangChain + Ollama + ChromaDB**